## Introduction

Data manipulation is a fundamental aspect of data analysis, and the choice of tools can significantly impact the efficiency and accuracy of the analysis. This notebook explores various data manipulation tasks using both SQL and pandas, two widely used tools in the field of data science. The tasks cover a spectrum of operations, ranging from simple queries to complex joins and aggregations. The primary goal is to showcase the versatility of SQL and pandas in handling diverse data manipulation scenarios.

In each task, we compare the results obtained using SQL and pandas, ensuring consistency between the two approaches. The dataset under consideration includes information about flights, planes, airlines, airports, and weather. By addressing each subtask and providing visualizations where applicable, we aim to demonstrate not only the technical proficiency in using SQL and pandas but also the practical considerations in choosing the right tool for specific data analysis tasks.

This report serves as a comprehensive guide to understanding the implementation of various data manipulation operations and the nuances involved in the SQL and pandas approaches. Through this exploration, we gain insights into the strengths and limitations of each tool, empowering data scientists to make informed decisions when working with diverse datasets.

### Loading Data into SQLite Database

In [1]:
import pandas as pd
import sqlite3
import gzip

# Establish a connection with a new SQLite database
conn = sqlite3.connect('nycflights13.db')

# Load CSV files into the SQLite database with skipping commented lines
flights = pd.read_csv('nycflights13_flights.csv.gz', compression='gzip', comment='#')
flights.to_sql('flights', conn, index=False, if_exists='replace')

airlines = pd.read_csv('nycflights13_airlines.csv.gz', compression='gzip', comment='#')
airlines.to_sql('airlines', conn, index=False, if_exists='replace')

airports = pd.read_csv('nycflights13_airports.csv.gz', compression='gzip', comment='#')
airports.to_sql('airports', conn, index=False, if_exists='replace')

planes = pd.read_csv('nycflights13_planes.csv.gz', compression='gzip', comment='#')
planes.to_sql('planes', conn, index=False, if_exists='replace')

weather = pd.read_csv('nycflights13_weather.csv.gz', compression='gzip', comment='#')
weather.to_sql('weather', conn, index=False, if_exists='replace')


26130

Purpose:
This code chunk establishes a connection to a new SQLite database and loads various CSV files into the corresponding tables in the database.

Summary/Discussion:
The dataset is now available in an SQLite database, making it accessible for both SQL and pandas manipulations in subsequent tasks.

### 1. SELECT DISTINCT engine FROM planes

In [2]:
# Equivalent Pandas Code
task1_sql = pd.read_sql_query("SELECT DISTINCT engine FROM planes", conn)

# Sort both DataFrames
task1_sql_sorted = task1_sql.sort_values(by='engine').reset_index(drop=True)

# solution using pandas — without SQL
task1_my = planes[['engine']].drop_duplicates()

# Sort both DataFrames
task1_my_sorted = task1_my.sort_values(by='engine').reset_index(drop=True)

# Perform the equality check
pd.testing.assert_frame_equal(task1_sql_sorted, task1_my_sorted)


Purpose:
In this task, we compare the distinct engine values obtained using SQL and pandas. The SQL query selects unique engine values from the 'planes' table, while the pandas solution achieves the same by selecting the 'engine' column and dropping duplicates.

Summary/Discussion:
The results from both SQL and pandas are equivalent, as confirmed by the assertion check. This task serves as a baseline for comparing the accuracy and consistency of SQL and pandas operations throughout the notebook.

### 2. SELECT DISTINCT type, engine FROM planes

In [3]:
# Equivalent Pandas Code
task2_sql = pd.read_sql_query("SELECT DISTINCT type, engine FROM planes", conn)

# Sort both DataFrames
task2_sql_sorted = task2_sql.sort_values(by=['type', 'engine']).reset_index(drop=True)

# solution using pandas — without SQL
task2_my = planes[['type', 'engine']].drop_duplicates()

# Sort both DataFrames
task2_my_sorted = task2_my.sort_values(by=['type', 'engine']).reset_index(drop=True)

# Perform the equality check
pd.testing.assert_frame_equal(task2_sql_sorted, task2_my_sorted)


Purpose:
This task involves comparing distinct combinations of 'type' and 'engine' obtained using SQL and pandas. The SQL query selects unique pairs of 'type' and 'engine' from the 'planes' table, while the pandas solution achieves the same by selecting the corresponding columns and dropping duplicates.

Summary/Discussion:
The results from both SQL and pandas are consistent, as confirmed by the assertion check. This task demonstrates the capability of pandas to replicate distinct values from multiple columns in SQL.

### 3. SELECT COUNT(*), engine FROM planes GROUP BY engine

In [4]:
# Equivalent Pandas Code
task3_sql = pd.read_sql_query("SELECT COUNT(*), engine FROM planes GROUP BY engine", conn)

# Sort both DataFrames
task3_sql_sorted = task3_sql.sort_values(by='engine').reset_index(drop=True)

# solution using pandas — without SQL
task3_my = planes.groupby('engine').size().reset_index(name='COUNT(*)')[['COUNT(*)', 'engine']]  # Adjust column order

# Sort both DataFrames
task3_my_sorted = task3_my.sort_values(by='engine').reset_index(drop=True)

# Perform the equality check
pd.testing.assert_frame_equal(task3_sql_sorted, task3_my_sorted)


Purpose:
In this task, the goal is to compare the counts of occurrences for each 'engine' obtained through SQL and pandas. The SQL query counts the number of rows for each 'engine' group in the 'planes' table, and the pandas solution achieves the same using the groupby and size functions.

Summary/Discussion:
The results from both SQL and pandas match, indicating that the count aggregation is consistent between the two approaches. This task illustrates how pandas can be employed to perform group-wise operations akin to SQL.

### 4. SELECT COUNT(*), engine, type FROM planes
### GROUP BY engine, type

In [5]:
# Equivalent Pandas Code
task4_sql = pd.read_sql_query("SELECT COUNT(*), engine, type FROM planes GROUP BY engine, type", conn)

# Sort both DataFrames
task4_sql_sorted = task4_sql.sort_values(by=['engine', 'type']).reset_index(drop=True)

# solution using pandas — without SQL
task4_my = planes.groupby(['engine', 'type']).size().reset_index(name='COUNT(*)')

# Reorder columns in task4_my to match task4_sql
task4_my = task4_my[['COUNT(*)', 'engine', 'type']]

# Sort both DataFrames
task4_my_sorted = task4_my.sort_values(by=['engine', 'type']).reset_index(drop=True)

# Perform the equality check
pd.testing.assert_frame_equal(task4_sql_sorted, task4_my_sorted)


Purpose:
This task involves counting the occurrences for each combination of 'engine' and 'type' in the 'planes' dataset using both SQL and pandas. The SQL query groups by both columns and counts the number of rows in each group. The pandas solution achieves the same using groupby and size functions.

Summary/Discussion:
The results from both SQL and pandas match, demonstrating consistency in counting occurrences for multiple columns across the two approaches. This showcases the versatility of pandas for such group-wise operations.

### 5. SELECT MIN(year), AVG(year), MAX(year), engine, manufacturer
### FROM planes
### GROUP BY engine, manufacturer

In [6]:
# Calculate the required statistics using pandas
task5_my = planes.groupby(['engine', 'manufacturer']).agg({'year': ['min', 'mean', 'max']}).reset_index()

# Rename the columns for consistency with SQL
task5_my.columns = ['engine', 'manufacturer', 'min_year', 'avg_year', 'max_year']

# Sort both DataFrames
task5_sql_sorted = pd.read_sql_query("""
    SELECT MIN(year) AS min_year, AVG(year) AS avg_year, MAX(year) AS max_year, engine, manufacturer
    FROM planes
    GROUP BY engine, manufacturer;
""", conn).sort_values(by=['engine', 'manufacturer']).reset_index(drop=True)

task5_my_sorted = task5_my.sort_values(by=['engine', 'manufacturer']).reset_index(drop=True)

# Perform the equality check
pd.testing.assert_frame_equal(task5_sql_sorted, task5_my_sorted, check_like=True)


Purpose:
Task 5 involves calculating the minimum, average, and maximum values of the 'year' column for each combination of 'engine' and 'manufacturer' in the 'planes' dataset. Both SQL and pandas are used for this calculation.

Summary/Discussion:
The results from both SQL and pandas match, demonstrating consistency in computing summary statistics for multiple columns across the two approaches. The check_like parameter ensures the comparison considers potential numerical variations. This task highlights pandas' ability to handle complex aggregations effectively.

### 6. SELECT * FROM planes WHERE speed IS NOT NULL

In [7]:
# Filter rows in the 'planes' DataFrame where 'speed' is not null
task6_my = planes[planes['speed'].notnull()]

# Reset index for consistent comparison
task6_my.reset_index(drop=True, inplace=True)

# Sort both DataFrames
task6_sql_sorted = pd.read_sql_query("""
    SELECT *
    FROM planes
    WHERE speed IS NOT NULL;
""", conn).sort_values(by=['speed']).reset_index(drop=True)

task6_my_sorted = task6_my.sort_values(by=['speed']).reset_index(drop=True)

# Perform the equality check
pd.testing.assert_frame_equal(task6_sql_sorted, task6_my_sorted, check_like=True)


Purpose:
Task 6 involves filtering rows in the 'planes' dataset where the 'speed' column is not null. Both pandas and SQL are used for this operation.

Summary/Discussion:
The results from both SQL and pandas match, indicating that the filtering operation has been performed consistently in both cases. The check_like parameter ensures a flexible comparison, accounting for potential numerical variations. This task demonstrates the equivalence of filtering operations in pandas and SQL.

### 7. SELECT tailnum FROM planes
### WHERE seats BETWEEN 150 AND 210 AND year >= 2011

In [8]:
# Filter rows in the 'planes' DataFrame based on conditions
task7_my = planes[(planes['seats'].between(150, 210)) & (planes['year'] >= 2011)][['tailnum']]

# Reset index for consistent comparison
task7_my.reset_index(drop=True, inplace=True)

# Sort both DataFrames
task7_sql_sorted = pd.read_sql_query("""
    SELECT tailnum
    FROM planes
    WHERE seats BETWEEN 150 AND 210 AND year >= 2011;
""", conn).sort_values(by=['tailnum']).reset_index(drop=True)

task7_my_sorted = task7_my.sort_values(by=['tailnum']).reset_index(drop=True)

# Perform the equality check
pd.testing.assert_frame_equal(task7_sql_sorted, task7_my_sorted)


Purpose:
Task 7 involves filtering rows in the 'planes' dataset based on specified conditions related to 'seats' and 'year'. Both pandas and SQL are used for this operation.

Summary/Discussion:
The results from both SQL and pandas match, indicating that the filtering operation has been performed consistently in both cases. This task demonstrates the equivalence of complex filtering conditions in pandas and SQL.

### 8. SELECT tailnum, manufacturer, seats FROM planes
### WHERE manufacturer IN ("BOEING", "AIRBUS", "EMBRAER") AND seats>390

In [9]:
# Filter rows in the 'planes' DataFrame based on conditions
manufacturers = ["BOEING", "AIRBUS", "EMBRAER"]
task8_my = planes[(planes['manufacturer'].isin(manufacturers)) & (planes['seats'] > 390)][['tailnum', 'manufacturer', 'seats']]

# Reset index for consistent comparison
task8_my.reset_index(drop=True, inplace=True)

# Sort both DataFrames
task8_sql_sorted = pd.read_sql_query("""
    SELECT tailnum, manufacturer, seats
    FROM planes
    WHERE manufacturer IN ("BOEING", "AIRBUS", "EMBRAER") AND seats > 390;
""", conn).sort_values(by=['tailnum']).reset_index(drop=True)

task8_my_sorted = task8_my.sort_values(by=['tailnum']).reset_index(drop=True)

# Perform the equality check
pd.testing.assert_frame_equal(task8_sql_sorted, task8_my_sorted)


Purpose:
Task 8 involves filtering rows in the 'planes' dataset based on multiple conditions, including a list of manufacturers and a minimum number of seats. Both pandas and SQL are used for this operation.

Summary/Discussion:
The results from both SQL and pandas match, indicating that the filtering operation has been performed consistently in both cases. This task demonstrates the equivalence of complex filtering conditions with multiple criteria in pandas and SQL.

### 9. SELECT DISTINCT year, seats FROM planes
### WHERE year >= 2012 ORDER BY year ASC, seats DESC

In [10]:
# Filter rows in the 'planes' DataFrame based on the condition
task9_my = planes[planes['year'] >= 2012][['year', 'seats']]

# Remove duplicate rows to simulate DISTINCT
task9_my.drop_duplicates(inplace=True)

# Sort the DataFrame based on 'year' in ascending order and 'seats' in descending order
task9_my.sort_values(by=['year', 'seats'], ascending=[True, False], inplace=True)

# Reset index for consistent comparison
task9_my.reset_index(drop=True, inplace=True)

# Sort the 'planes' DataFrame from SQL query
task9_sql_sorted = pd.read_sql_query("""
    SELECT DISTINCT year, seats
    FROM planes
    WHERE year >= 2012
    ORDER BY year ASC, seats DESC;
""", conn).sort_values(by=['year', 'seats'], ascending=[True, False]).reset_index(drop=True)

# Perform the equality check
pd.testing.assert_frame_equal(task9_sql_sorted, task9_my)


Purpose:
Task 9 involves filtering rows in the 'planes' dataset based on a condition (year >= 2012) and selecting specific columns ('year' and 'seats'). Both pandas and SQL are used for this operation.

Summary/Discussion:
The results from both SQL and pandas match, indicating that the filtering and column selection operations have been performed consistently in both cases. This task demonstrates the equivalence of filtering rows, selecting columns, and using DISTINCT in pandas and SQL.

### 10. SELECT DISTINCT year, seats FROM planes
### WHERE year >= 2012 ORDER BY seats DESC, year ASC

In [11]:
# Filter rows in the 'planes' DataFrame based on the condition
task10_my = planes[planes['year'] >= 2012][['year', 'seats']]

# Remove duplicate rows to simulate DISTINCT
task10_my.drop_duplicates(inplace=True)

# Sort the DataFrame based on 'seats' in descending order and 'year' in ascending order
task10_my.sort_values(by=['seats', 'year'], ascending=[False, True], inplace=True)

# Reset index for consistent comparison
task10_my.reset_index(drop=True, inplace=True)

# Sort the 'planes' DataFrame from SQL query
task10_sql_sorted = pd.read_sql_query("""
    SELECT DISTINCT year, seats
    FROM planes
    WHERE year >= 2012
    ORDER BY seats DESC, year ASC;
""", conn).sort_values(by=['seats', 'year'], ascending=[False, True]).reset_index(drop=True)

# Perform the equality check
pd.testing.assert_frame_equal(task10_sql_sorted, task10_my)


Purpose:
Task 10 involves filtering rows in the 'planes' dataset based on a condition (year >= 2012), selecting specific columns ('year' and 'seats'), removing duplicates, and sorting the DataFrame based on 'seats' in descending order and 'year' in ascending order. Both pandas and SQL are used for this operation.

Summary/Discussion:
The results from both SQL and pandas match, indicating that the filtering, column selection, duplicate removal, and sorting operations have been performed consistently in both cases. This task demonstrates the equivalence of these operations in pandas and SQL.


### 11. SELECT manufacturer, COUNT(*) FROM planes
### WHERE seats > 200 GROUP BY manufacturer

In [12]:
# Filter rows in the 'planes' DataFrame based on the condition
task11_my = planes[planes['seats'] > 200]

# Group by 'manufacturer' and count the occurrences
task11_my = task11_my.groupby('manufacturer').size().reset_index(name='count')

# Sort the DataFrame based on 'count' in descending order and 'manufacturer' in ascending order
task11_my.sort_values(by=['count', 'manufacturer'], ascending=[False, True], inplace=True)

# Reset index for consistent comparison
task11_my.reset_index(drop=True, inplace=True)

# Sort the 'planes' DataFrame from SQL query
task11_sql_sorted = pd.read_sql_query("""
    SELECT manufacturer, COUNT(*) AS count
    FROM planes
    WHERE seats > 200
    GROUP BY manufacturer;
""", conn).sort_values(by=['count', 'manufacturer'], ascending=[False, True]).reset_index(drop=True)

# Perform the equality check
pd.testing.assert_frame_equal(task11_sql_sorted, task11_my)


Purpose:
Task 11 involves filtering rows in the 'planes' dataset based on a condition (seats > 200), grouping by 'manufacturer,' counting occurrences, and sorting the result by count in descending order and 'manufacturer' in ascending order. Both pandas and SQL are used for this operation.

Summary/Discussion:
The results from both SQL and pandas match, indicating that the filtering, grouping, counting, and sorting operations have been performed consistently in both cases. This task demonstrates the equivalence of these operations in pandas and SQL.

### 12. SELECT manufacturer, COUNT(*) FROM planes
### GROUP BY manufacturer HAVING COUNT(*) > 10

In [13]:
# Group by 'manufacturer' and count the occurrences
task12_my = planes.groupby('manufacturer').size().reset_index(name='count')

# Filter rows where the count is greater than 10
task12_my = task12_my[task12_my['count'] > 10]

# Sort the DataFrame based on 'count' in descending order and 'manufacturer' in ascending order
task12_my.sort_values(by=['count', 'manufacturer'], ascending=[False, True], inplace=True)

# Reset index for consistent comparison
task12_my.reset_index(drop=True, inplace=True)

# Sort the 'planes' DataFrame from SQL query
task12_sql_sorted = pd.read_sql_query("""
    SELECT manufacturer, COUNT(*) AS count
    FROM planes
    GROUP BY manufacturer
    HAVING COUNT(*) > 10;
""", conn).sort_values(by=['count', 'manufacturer'], ascending=[False, True]).reset_index(drop=True)

# Perform the equality check
pd.testing.assert_frame_equal(task12_sql_sorted, task12_my)


Purpose:
Task 12 involves grouping the 'planes' dataset by 'manufacturer,' counting occurrences, filtering rows where the count is greater than 10, and sorting the result by count in descending order and 'manufacturer' in ascending order. Both pandas and SQL are used for this operation.

Summary/Discussion:
The results from both SQL and pandas match, indicating that the grouping, counting, filtering, and sorting operations have been performed consistently in both cases. This task demonstrates the equivalence of these operations in pandas and SQL.


### 13. SELECT manufacturer, COUNT(*) FROM planes
### WHERE seats > 200 GROUP BY manufacturer HAVING COUNT(*) > 10

In [14]:
# Filter rows where seats > 200
task13_my_filtered = planes[planes['seats'] > 200]

# Group by 'manufacturer' and count the occurrences
task13_my = task13_my_filtered.groupby('manufacturer').size().reset_index(name='count')

# Filter rows where the count is greater than 10
task13_my = task13_my[task13_my['count'] > 10]

# Sort the DataFrame based on 'count' in descending order and 'manufacturer' in ascending order
task13_my.sort_values(by=['count', 'manufacturer'], ascending=[False, True], inplace=True)

# Reset index for consistent comparison
task13_my.reset_index(drop=True, inplace=True)

# Sort the 'planes' DataFrame from SQL query
task13_sql_sorted = pd.read_sql_query("""
    SELECT manufacturer, COUNT(*) AS count
    FROM planes
    WHERE seats > 200
    GROUP BY manufacturer
    HAVING COUNT(*) > 10;
""", conn).sort_values(by=['count', 'manufacturer'], ascending=[False, True]).reset_index(drop=True)

# Perform the equality check
pd.testing.assert_frame_equal(task13_sql_sorted, task13_my)


Purpose:
Task 13 involves filtering rows in the 'planes' dataset where 'seats' > 200, grouping by 'manufacturer,' counting occurrences, filtering rows where the count is greater than 10, and sorting the result by count in descending order and 'manufacturer' in ascending order. Both pandas and SQL are used for this operation.

Summary/Discussion:
The results from both SQL and pandas match, indicating that the filtering, grouping, counting, and sorting operations have been performed consistently in both cases. This task demonstrates the equivalence of these operations in pandas and SQL.

### 14. SELECT manufacturer, COUNT(*) AS howmany
### FROM planes
### GROUP BY manufacturer
### ORDER BY howmany DESC LIMIT 10

In [15]:
# Group by 'manufacturer' and count the occurrences
task14_my = planes.groupby('manufacturer').size().reset_index(name='howmany')

# Sort the DataFrame based on 'howmany' in descending order and 'manufacturer' in ascending order
task14_my.sort_values(by=['howmany', 'manufacturer'], ascending=[False, True], inplace=True)

# Retrieve the top 10 rows
task14_my = task14_my.head(10)

# Reset index for consistent comparison
task14_my.reset_index(drop=True, inplace=True)

# Sort the 'planes' DataFrame from SQL query
task14_sql_sorted = pd.read_sql_query("""
    SELECT manufacturer, COUNT(*) AS howmany
    FROM planes
    GROUP BY manufacturer
    ORDER BY howmany DESC
    LIMIT 10;
""", conn).sort_values(by=['howmany', 'manufacturer'], ascending=[False, True]).reset_index(drop=True)

# Perform the equality check
pd.testing.assert_frame_equal(task14_sql_sorted, task14_my)


Purpose:
Task 14 involves grouping the 'planes' dataset by 'manufacturer,' counting occurrences, sorting the result by count in descending order and 'manufacturer' in ascending order, and retrieving the top 10 rows. Both pandas and SQL are used for this operation.

Summary/Discussion:
The results from both SQL and pandas match, indicating that the grouping, counting, sorting, and limiting operations have been performed consistently in both cases. This task demonstrates the equivalence of these operations in pandas and SQL.

### 15. SELECT
### flights.*,
### planes.year AS plane_year,
### planes.speed AS plane_speed,
### planes.seats AS plane_seats
### FROM flights LEFT JOIN planes ON flights.tailnum=planes.tailnum

In [16]:
# Perform a left merge on 'flights' and 'planes' DataFrames
merged_df = pd.merge(flights, planes[['tailnum', 'year', 'speed', 'seats']], on='tailnum', how='left')

# Select the columns you need and rename them
task15_my = merged_df[['year_x', 'month', 'day', 'dep_time', 'sched_dep_time', 'dep_delay',
                        'arr_time', 'sched_arr_time', 'arr_delay', 'carrier', 'flight',
                        'tailnum', 'origin', 'dest', 'air_time', 'distance', 'hour', 'minute',
                        'time_hour', 'year_y', 'speed', 'seats']].rename(
                            columns={'year_x': 'flights_year', 'year_y': 'plane_year', 'speed': 'plane_speed', 'seats': 'plane_seats'})

# Sort both DataFrames based on a chosen column for consistent comparison
task15_sql_sorted = pd.read_sql_query("""
    SELECT
        flights.*,
        planes.year AS plane_year,
        planes.speed AS plane_speed,
        planes.seats AS plane_seats
    FROM flights
    LEFT JOIN planes ON flights.tailnum=planes.tailnum;
""", conn).sort_values(by='tailnum').reset_index(drop=True)

# Rename the columns in task15_sql_sorted to match the column names in task15_my
task15_sql_sorted = task15_sql_sorted.rename(
    columns={'year': 'flights_year', 'speed': 'plane_speed', 'seats': 'plane_seats'}
)

task15_my_sorted = task15_my.sort_values(by='tailnum').reset_index(drop=True)

# Perform the equality check
pd.testing.assert_frame_equal(task15_sql_sorted, task15_my_sorted)


C:\Users\mailv\AppData\Local\Temp\ipykernel_38292\3097387301.py:30: FutureWarning: Mismatched null-like values None and nan found. In a future version, pandas equality-testing functions (e.g. assert_frame_equal) will consider these not-matching and raise.
  pd.testing.assert_frame_equal(task15_sql_sorted, task15_my_sorted)


Purpose:
Task 15 involves performing a left merge on 'flights' and 'planes' DataFrames based on the 'tailnum' column. After merging, specific columns are selected, and their names are renamed for consistency. Both pandas and SQL are used for this operation.

Summary/Discussion:
The results from both SQL and pandas match, indicating that the left merge, column selection, and renaming operations have been performed consistently in both cases. This task demonstrates the equivalence of these operations in pandas and SQL.

### 16. SELECT planes.*, airlines.* FROM
### (SELECT DISTINCT carrier, tailnum FROM flights) AS cartail
### INNER JOIN planes ON cartail.tailnum=planes.tailnum
### INNER JOIN airlines ON cartail.carrier=airlines.carrier

In [17]:
# SQL query
query = """
SELECT planes.*, airlines.* FROM
(SELECT DISTINCT carrier, tailnum FROM flights) AS cartail
INNER JOIN planes ON cartail.tailnum=planes.tailnum
INNER JOIN airlines ON cartail.carrier=airlines.carrier;
"""

# Reading SQL query result into a pandas DataFrame
task16_sql = pd.read_sql_query(query, conn)

# Sort the result for consistent comparison
task16_sql_sorted = task16_sql.sort_values(by=['tailnum', 'carrier']).reset_index(drop=True)

# pandas solution without SQL
# Assuming DataFrame 'flights' with columns 'carrier' and 'tailnum'
cartail = flights[['carrier', 'tailnum']].drop_duplicates()

# Assuming DataFrames 'planes' and 'airlines' with columns 'tailnum' and 'carrier' respectively
result_df = pd.merge(cartail, planes, on='tailnum', how='inner')
result_df = pd.merge(result_df, airlines, on='carrier', how='inner')

# Reorder columns to match the SQL result
result_df = result_df[['tailnum', 'year', 'type', 'manufacturer', 'model', 'engines', 'seats',
                       'speed', 'engine', 'carrier', 'name']]

# Sort the result for consistent comparison
result_df_sorted = result_df.sort_values(by=['tailnum', 'carrier']).reset_index(drop=True)

# Perform the equality check
pd.testing.assert_frame_equal(task16_sql_sorted, result_df_sorted)


Purpose:
Task 16 involves performing SQL joins in pandas without using SQL. The query retrieves unique pairs of 'carrier' and 'tailnum' from the 'flights' DataFrame, then performs inner joins with the 'planes' and 'airlines' DataFrames based on the specified conditions.

Summary/Discussion:
The results from both SQL and pandas match, indicating that the SQL joins have been successfully replicated using pandas without relying on SQL. This task demonstrates the equivalence of these operations in both approaches.

### 17. SELECT
### flights2.*,
### atemp,
### ahumid
### FROM (
### SELECT * FROM flights WHERE origin='EWR'
### ) AS flights2
### LEFT JOIN (
### SELECT
### year, month, day,
### AVG(temp) AS atemp,
### AVG(humid) AS ahumid
### FROM weather
### WHERE origin='EWR'
### GROUP BY year, month, day
### ) AS weather2
### ON flights2.year=weather2.year
### AND flights2.month=weather2.month
### AND flights2.day=weather2.day

In [18]:
# SQL query
query = """
SELECT
    flights2.*,
    atemp AS temp,
    ahumid AS humid
FROM (
    SELECT * FROM flights WHERE origin='EWR'
) AS flights2
LEFT JOIN (
    SELECT
        year, month, day,
        AVG(temp) AS atemp,
        AVG(humid) AS ahumid
    FROM weather
    WHERE origin='EWR'
    GROUP BY year, month, day
) AS weather2
ON flights2.year=weather2.year
AND flights2.month=weather2.month
AND flights2.day=weather2.day;
"""

# Reading SQL query result into a pandas DataFrame
task17_sql = pd.read_sql_query(query, conn)

# Sort the result for consistent comparison
task17_sql_sorted = task17_sql.sort_values(by=['year', 'month', 'day']).reset_index(drop=True)

# pandas solution without SQL
# Assuming DataFrames 'flights' and 'weather' with the required columns
flights2 = flights[flights['origin'] == 'EWR']
weather2 = weather[weather['origin'] == 'EWR'].groupby(['year', 'month', 'day']).agg({'temp': 'mean', 'humid': 'mean'}).reset_index()

# Select only the necessary columns and rename them to match the SQL query
result_df = pd.merge(flights2, weather2[['year', 'month', 'day', 'temp', 'humid']], on=['year', 'month', 'day'], how='left')

# Perform the equality check
pd.testing.assert_frame_equal(task17_sql_sorted, result_df.sort_values(by=['year', 'month', 'day']).reset_index(drop=True))


C:\Users\mailv\AppData\Local\Temp\ipykernel_38292\525025734.py:39: FutureWarning: Mismatched null-like values None and nan found. In a future version, pandas equality-testing functions (e.g. assert_frame_equal) will consider these not-matching and raise.
  pd.testing.assert_frame_equal(task17_sql_sorted, result_df.sort_values(by=['year', 'month', 'day']).reset_index(drop=True))


Purpose:
Task 17 involves performing SQL joins with aggregation in pandas without using SQL. The query retrieves flights originating from 'EWR' and performs a left join with aggregated weather data based on 'year', 'month', and 'day'.

Summary/Discussion:
The results from both SQL and pandas match, indicating that the SQL joins with aggregation have been successfully replicated using pandas without relying on SQL. This task demonstrates the equivalence of these operations in both approaches.

### Conclusion

In this report, I addressed various tasks involving SQL queries and their equivalent implementations using pandas in a Jupyter notebook. The tasks covered a range of operations, including basic selects, aggregations, filtering, and joins. For each task, the provided SQL query was translated into pandas code, and the results were compared to ensure equivalence.

The pandas code successfully replicated the SQL functionality across all tasks, validating the flexibility and power of pandas as a data manipulation tool. Throughout the analysis, the results from both approaches were consistently identical, indicating a successful translation of SQL queries into pandas operations.

For future extensions, one could explore optimizing the pandas code for larger datasets or enhancing the analysis with additional features. Additionally, this notebook serves as a valuable reference for users transitioning between SQL and pandas, showcasing practical examples of how to achieve similar results in both environments.

Overall, this exercise not only demonstrated the versatility of pandas for data manipulation but also provided a comprehensive guide for users familiar with SQL to leverage their skills in a pandas-based data analysis environment.
